In [1]:
from dask import optimize
from timelapse_tools import daread
daread?

Signature: daread(img: Union[str, pathlib.Path, aicspylibczi.CziFile.CziFile]) -> <module 'dask.array' from '/home/maxfield/miniconda3/envs/timelapse_tools/lib/python3.7/site-packages/dask/array/__init__.py'>
Docstring: <no docstring>
File:      ~/active/cell/timelapse_tools/timelapse_tools/utils/czi_reading.py
Type:      function


In [2]:
t, t_dims = daread("timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi")
print(t_dims)
t

/home/maxfield/active/cell/timelapse_tools/timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi
STCZYX


,Array,Chunk
Bytes,5.77 MB,1.15 MB
Shape,"(1, 5, 1, 1, 624, 924)","(1, 1, 1, 1, 624, 924)"
Count,20 Tasks,5 Chunks
Type,uint16,numpy.ndarray


In [3]:
optimize(t[0, 0, 0, 0, :])[0]

dask.array<getitem, shape=(624, 924), dtype=uint16, chunksize=(624, 924), chunktype=numpy.ndarray>

In [4]:
s, s_dims = daread("timelapse_tools/tests/data/s_None_t_5_c_1_z_None.czi")
print(s_dims)
s

/home/maxfield/active/cell/timelapse_tools/timelapse_tools/tests/data/s_None_t_5_c_1_z_None.czi
TCYX


dask.array<concatenate, shape=(5, 1, 1248, 1848), dtype=uint16, chunksize=(1, 1, 1248, 1848), chunktype=numpy.ndarray>

In [5]:
optimize(s[0, 0, :])[0]

dask.array<getitem, shape=(1248, 1848), dtype=uint16, chunksize=(1248, 1848), chunktype=numpy.ndarray>

In [ ]:
p, p_dims = daread("/allen/aics/assay-dev/MicroscopyData/Melissa/2018/20180211/20180211_M04_001.czi")
print(p_dims)
p

In [ ]:
optimize(p[0, 0, 0, 0, :])[0]